In [1]:
import pandas as pd
import numpy as np
final_pricing_df = pd.read_csv("cost_margin_bucket.csv")
Market_Price_Competitor_Data  = pd.read_csv("Market_Price_Competitor_Data.csv")
Historical_Conversion_Data = pd.read_csv("Historical_Conversion_Data.csv")
Returns_Pricing_Signal = pd.read_csv("Returns_Pricing_Signal.csv")
Inventory_Health = pd.read_csv("Inventory_Health.csv")   
Aggregrated_ads_Data = pd.read_csv("Aggregrated_ads_Data.csv")

In [9]:
dataset = [final_pricing_df,Market_Price_Competitor_Data,Historical_Conversion_Data,Returns_Pricing_Signal,
           Inventory_Health,Aggregrated_ads_Data]
for col in dataset:
    print(col.head())
    print("-" * 50)


     SKU                       Product_description Product Role  unit_cost  \
0  MN-01  Rectangle Tray -14 x 10 Inch - Pk of  25         Core      32.30   
1  MN-02          Rectangle Tray -12x10 - Pk of 25         Core      26.60   
2  MN-03         Oval Tray -15x10 Inch - Pk of  25         Core      30.03   
3  MN-04          Oval Tray - 13x9 Inch - pk of 25         Core      25.94   
4  MN-05     Christmas Tray - 17x12 Inch - Pk of 5     Seasonal      25.77   

   min_price  target_price  Current_Price  margin_violation  \
0    40.3750     49.692308           38.9              True   
1    33.2500     40.923077           33.9             False   
2    37.5375     46.200000           34.9              True   
3    32.4250     39.907692           29.9              True   
4    34.3600     46.854545           21.9              True   

   loss_amount_per_unit  loss_making  
0                  0.00        False  
1                  0.00        False  
2                  0.00        Fals

In [12]:
pricing_base = final_pricing_df[
    [
        "SKU",
        "unit_cost",
        "min_price",
        "target_price",
        "Current_Price",
        "margin_violation",
        "loss_making",
    ]
]


market_base = Market_Price_Competitor_Data[
    [
        "SKU",
        "resolved_avg_Competitor_price",
        "Lowest_Competitor_Price",
        "resolved_highest_competitor_price",
        "market_position",
    ]
]



demand_base = Historical_Conversion_Data[
    [
        "SKU",
        "Units Ordered",
        "Demand Strength",
        "Conversion Rate",
        "Revenue per Session",
    ]
]



returns_base = Returns_Pricing_Signal[
    [
        "SKU",
        "return_rate_90d",
        "return_risk",
        "demand_return_regime",
    ]
]



inventory_base = Inventory_Health[
    [
        "SKU",
        "days-of-supply",
        "inventory_status",
        "velocity_trend",
        "clearance_candidate",
        "sell-through",
        "pricing_strategy_inventory",
    ]
]



ads_base = Aggregrated_ads_Data[
    [
        "SKU",
        "ads_efficiency",
        "ad_dependency_flag",
        "pricing_strategy_ads",
    ]
]



In [13]:
master_df = (
    pricing_base
    .merge(market_base, on="SKU", how="left")
    .merge(demand_base, on="SKU", how="left")
    .merge(returns_base, on="SKU", how="left")
    .merge(inventory_base, on="SKU", how="left")
    .merge(ads_base, on="SKU", how="left")
)


In [15]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   SKU                                50 non-null     object 
 1   unit_cost                          50 non-null     float64
 2   min_price                          50 non-null     float64
 3   target_price                       50 non-null     float64
 4   Current_Price                      50 non-null     float64
 5   margin_violation                   50 non-null     bool   
 6   loss_making                        50 non-null     bool   
 7   resolved_avg_Competitor_price      50 non-null     float64
 8   Lowest_Competitor_Price            50 non-null     float64
 9   resolved_highest_competitor_price  50 non-null     float64
 10  market_position                    50 non-null     object 
 11  Units Ordered                      50 non-null     float64
 

In [16]:
master_df.columns

Index(['SKU', 'unit_cost', 'min_price', 'target_price', 'Current_Price',
       'margin_violation', 'loss_making', 'resolved_avg_Competitor_price',
       'Lowest_Competitor_Price', 'resolved_highest_competitor_price',
       'market_position', 'Units Ordered', 'Demand Strength',
       'Conversion Rate', 'Revenue per Session', 'return_rate_90d',
       'return_risk', 'demand_return_regime', 'days-of-supply',
       'inventory_status', 'velocity_trend', 'clearance_candidate',
       'sell-through', 'pricing_strategy_inventory', 'ads_efficiency',
       'ad_dependency_flag', 'pricing_strategy_ads'],
      dtype='object')

In [17]:
# Hard Price Bounds (NON-NEGOTIABLE)
master_df["price_lower_bound"] = master_df[
    ["min_price", "Lowest_Competitor_Price"]
].max(axis="columns")

master_df["price_upper_bound"] = master_df["resolved_highest_competitor_price"]
master_df

,SKU,unit_cost,min_price,target_price,Current_Price,margin_violation,loss_making,resolved_avg_Competitor_price,Lowest_Competitor_Price,resolved_highest_competitor_price,...,inventory_status,velocity_trend,clearance_candidate,sell-through,pricing_strategy_inventory,ads_efficiency,ad_dependency_flag,pricing_strategy_ads,price_lower_bound,price_upper_bound
0,MN-01,32.30,40.375000,49.692308,38.90,True,False,36.900,31.95,44.900,...,Healthy,Stable / Growing,No,1.33,Increase Price,Moderate Ads,Normal,Hold Price,40.375000,44.900
1,MN-02,26.60,33.250000,40.923077,33.90,False,False,32.500,27.90,39.950,...,Healthy,Stable / Growing,No,1.56,Small Increase Price,Efficient Ads,Normal,Allow Price Increase,33.250000,39.950
2,MN-03,30.03,37.537500,46.200000,34.90,True,False,35.200,29.99,42.500,...,Healthy,Stable / Growing,No,1.16,Hold Price,Efficient Ads,Normal,Allow Price Increase,37.537500,42.500
3,MN-04,25.94,32.425000,39.907692,29.90,True,False,28.400,24.50,35.900,...,Overstocked,Stable / Growing,No,1.65,Hold Price (Absorb Inventory),Inefficient Ads,High Ad Dependency,Reduce Price or Improve Listing,32.425000,35.900
4,MN-05,25.77,34.360000,46.854545,21.90,True,True,23.900,18.95,27.485,...,Overstocked,Stable / Growing,No,1.47,Hold Price (Absorb Inventory),Moderate Ads,High Ad Dependency,Hold Price,34.360000,27.485
5,MN-06,14.37,19.160000,26.127273,14.95,True,False,15.800,12.99,21.500,...,Low Inventory,Stable / Growing,No,1.23,Hold Price,Inefficient Ads,High Ad Dependency,Reduce Price or Improve Listing,19.160000,21.500
6,MN-07,29.87,37.337500,45.953846,34.90,True,False,33.500,28.90,41.950,...,Healthy,Stable / Growing,No,1.25,Increase Price,Efficient Ads,Normal,Allow Price Increase,37.337500,41.950
7,MN-08,26.11,34.813333,47.472727,29.90,True,False,31.200,25.95,39.900,...,Overstocked,Demand Slowing,Yes,0.37,Reduce Price (Clearance),Moderate Ads,Normal,Hold Price,34.813333,39.900
8,MN-09,14.92,19.893333,27.127273,14.90,True,True,15.600,11.99,22.900,...,Healthy,Demand Slowing,No,0.36,Hold Price,Moderate Ads,High Ad Dependency,Hold Price,19.893333,22.900
9,MN-10,17.40,21.750000,26.769231,19.90,True,False,20.500,16.95,26.900,...,Overstocked,Stable / Growing,No,1.17,Hold Price,Moderate Ads,Normal,Hold Price,21.750000,26.900


In [20]:
master_df["demand_return_regime"].value_counts()

demand_return_regime
Mixed / Watch                  33
Weak Demand & High Returns      7
False Demand (High Returns)     6
Low Demand but Good Quality     3
True Demand (Pricing Power)     1
Name: count, dtype: int64

In [21]:
# Setting Demand–Return score
demand_return_score_map = {
    "True Demand (Pricing Power)": 1.0, # Strong demand + low returns --> safe to increase price
    "Low Demand but Good Quality": -0.3, # Customers are satisfied(low return rates) but demand is weak --> small price support
    "Mixed / Watch": 0.0,                # Conflicting signals --> no price change
    "False Demand (High Returns)": -1.0, # Strong demand but high return rates --> pricing power is misleading
    "Weak Demand & High Returns": -1.5, # Worst case -- > reduce price or de-prioritize SKU
}

master_df["demand_return_score"] = (
    master_df["demand_return_regime"]
    .map(demand_return_score_map)
)


In [23]:
master_df[["demand_return_regime","demand_return_score"]]

,demand_return_regime,demand_return_score
0,Mixed / Watch,0.0
1,Mixed / Watch,0.0
2,Mixed / Watch,0.0
3,False Demand (High Returns),-1.0
4,Low Demand but Good Quality,-0.3
5,Low Demand but Good Quality,-0.3
6,Mixed / Watch,0.0
7,Weak Demand & High Returns,-1.5
8,Weak Demand & High Returns,-1.5
9,Weak Demand & High Returns,-1.5


In [24]:
# Setting Inventory score
inventory_score_map = {
    "Increase Price": 0.8,
    "Small Increase Price": 0.4,
    "Hold Price": 0.0,
    "Hold Price (Absorb Inventory)": 0.0,
    "Reduce Price (Clearance)": -1.2,
    "Freeze Pricing,Fix Availability": -0.5,
}

master_df["inventory_score"] = (
    master_df["pricing_strategy_inventory"]
    .map(inventory_score_map)
    .fillna(0)
)


In [26]:
ads_score_map = {
    "Allow Price Increase": 0.5,
    "Hold Price": 0.0,
    "Reduce Price or Improve Listing": -0.7,
    "Freeze Price Increase": -0.5,
}

master_df["ads_score"] = (
    master_df["pricing_strategy_ads"]
    .map(ads_score_map)
    .fillna(0)
)


In [27]:
master_df

,SKU,unit_cost,min_price,target_price,Current_Price,margin_violation,loss_making,resolved_avg_Competitor_price,Lowest_Competitor_Price,resolved_highest_competitor_price,...,sell-through,pricing_strategy_inventory,ads_efficiency,ad_dependency_flag,pricing_strategy_ads,price_lower_bound,price_upper_bound,demand_return_score,inventory_score,ads_score
0,MN-01,32.30,40.375000,49.692308,38.90,True,False,36.900,31.95,44.900,...,1.33,Increase Price,Moderate Ads,Normal,Hold Price,40.375000,44.900,0.0,0.8,0.0
1,MN-02,26.60,33.250000,40.923077,33.90,False,False,32.500,27.90,39.950,...,1.56,Small Increase Price,Efficient Ads,Normal,Allow Price Increase,33.250000,39.950,0.0,0.4,0.5
2,MN-03,30.03,37.537500,46.200000,34.90,True,False,35.200,29.99,42.500,...,1.16,Hold Price,Efficient Ads,Normal,Allow Price Increase,37.537500,42.500,0.0,0.0,0.5
3,MN-04,25.94,32.425000,39.907692,29.90,True,False,28.400,24.50,35.900,...,1.65,Hold Price (Absorb Inventory),Inefficient Ads,High Ad Dependency,Reduce Price or Improve Listing,32.425000,35.900,-1.0,0.0,-0.7
4,MN-05,25.77,34.360000,46.854545,21.90,True,True,23.900,18.95,27.485,...,1.47,Hold Price (Absorb Inventory),Moderate Ads,High Ad Dependency,Hold Price,34.360000,27.485,-0.3,0.0,0.0
5,MN-06,14.37,19.160000,26.127273,14.95,True,False,15.800,12.99,21.500,...,1.23,Hold Price,Inefficient Ads,High Ad Dependency,Reduce Price or Improve Listing,19.160000,21.500,-0.3,0.0,-0.7
6,MN-07,29.87,37.337500,45.953846,34.90,True,False,33.500,28.90,41.950,...,1.25,Increase Price,Efficient Ads,Normal,Allow Price Increase,37.337500,41.950,0.0,0.8,0.5
7,MN-08,26.11,34.813333,47.472727,29.90,True,False,31.200,25.95,39.900,...,0.37,Reduce Price (Clearance),Moderate Ads,Normal,Hold Price,34.813333,39.900,-1.5,-1.2,0.0
8,MN-09,14.92,19.893333,27.127273,14.90,True,True,15.600,11.99,22.900,...,0.36,Hold Price,Moderate Ads,High Ad Dependency,Hold Price,19.893333,22.900,-1.5,0.0,0.0
9,MN-10,17.40,21.750000,26.769231,19.90,True,False,20.500,16.95,26.900,...,1.17,Hold Price,Moderate Ads,Normal,Hold Price,21.750000,26.900,-1.5,0.0,0.0


## FINAL PRICING SCORE


In [28]:
master_df["final_pricing_score"] = (
    master_df["demand_return_score"]
    + master_df["inventory_score"]
    + master_df["ads_score"]
)


In [29]:
def price_adjustment_pct(score):
    if score >= 1.5:
        return 0.10
    if score >= 0.5:
        return 0.05
    if score <= -1.5:
        return -0.15
    if score <= -0.5:
        return -0.08
    return 0.0

master_df["price_adjustment_pct"] = (
    master_df["final_pricing_score"]
    .apply(price_adjustment_pct)
)


In [30]:
master_df["raw_recommended_price"] = (
    master_df["Current_Price"]
    * (1 + master_df["price_adjustment_pct"])
)

master_df["recommended_price"] = (
    master_df["raw_recommended_price"]
    .clip(
        lower=master_df["price_lower_bound"],
        upper=master_df["price_upper_bound"]
    )
)


In [31]:
master_df

,SKU,unit_cost,min_price,target_price,Current_Price,margin_violation,loss_making,resolved_avg_Competitor_price,Lowest_Competitor_Price,resolved_highest_competitor_price,...,pricing_strategy_ads,price_lower_bound,price_upper_bound,demand_return_score,inventory_score,ads_score,final_pricing_score,price_adjustment_pct,raw_recommended_price,recommended_price
0,MN-01,32.30,40.375000,49.692308,38.90,True,False,36.900,31.95,44.900,...,Hold Price,40.375000,44.900,0.0,0.8,0.0,0.8,0.05,40.845,40.845000
1,MN-02,26.60,33.250000,40.923077,33.90,False,False,32.500,27.90,39.950,...,Allow Price Increase,33.250000,39.950,0.0,0.4,0.5,0.9,0.05,35.595,35.595000
2,MN-03,30.03,37.537500,46.200000,34.90,True,False,35.200,29.99,42.500,...,Allow Price Increase,37.537500,42.500,0.0,0.0,0.5,0.5,0.05,36.645,37.537500
3,MN-04,25.94,32.425000,39.907692,29.90,True,False,28.400,24.50,35.900,...,Reduce Price or Improve Listing,32.425000,35.900,-1.0,0.0,-0.7,-1.7,-0.15,25.415,32.425000
4,MN-05,25.77,34.360000,46.854545,21.90,True,True,23.900,18.95,27.485,...,Hold Price,34.360000,27.485,-0.3,0.0,0.0,-0.3,0.00,21.900,34.360000
5,MN-06,14.37,19.160000,26.127273,14.95,True,False,15.800,12.99,21.500,...,Reduce Price or Improve Listing,19.160000,21.500,-0.3,0.0,-0.7,-1.0,-0.08,13.754,19.160000
6,MN-07,29.87,37.337500,45.953846,34.90,True,False,33.500,28.90,41.950,...,Allow Price Increase,37.337500,41.950,0.0,0.8,0.5,1.3,0.05,36.645,37.337500
7,MN-08,26.11,34.813333,47.472727,29.90,True,False,31.200,25.95,39.900,...,Hold Price,34.813333,39.900,-1.5,-1.2,0.0,-2.7,-0.15,25.415,34.813333
8,MN-09,14.92,19.893333,27.127273,14.90,True,True,15.600,11.99,22.900,...,Hold Price,19.893333,22.900,-1.5,0.0,0.0,-1.5,-0.15,12.665,19.893333
9,MN-10,17.40,21.750000,26.769231,19.90,True,False,20.500,16.95,26.900,...,Hold Price,21.750000,26.900,-1.5,0.0,0.0,-1.5,-0.15,16.915,21.750000


In [52]:

(master_df.head(2))

,SKU,unit_cost,min_price,target_price,Current_Price,margin_violation,loss_making,resolved_avg_Competitor_price,Lowest_Competitor_Price,resolved_highest_competitor_price,...,pricing_strategy_ads,price_lower_bound,price_upper_bound,demand_return_score,inventory_score,ads_score,final_pricing_score,price_adjustment_pct,raw_recommended_price,recommended_price
0,MN-01,32.3,40.375,49.692308,38.9,True,False,36.9,31.95,44.90,...,Hold Price,40.375,44.90,0.0,0.8,0.0,0.8,0.05,40.845,40.845
1,MN-02,26.6,33.250,40.923077,33.9,False,False,32.5,27.90,39.95,...,Allow Price Increase,33.250,39.95,0.0,0.4,0.5,0.9,0.05,35.595,35.595


In [53]:
master_df.to_csv("Final_Price_strategy.csv",index = False)

In [54]:
master_df.columns

Index(['SKU', 'unit_cost', 'min_price', 'target_price', 'Current_Price',
       'margin_violation', 'loss_making', 'resolved_avg_Competitor_price',
       'Lowest_Competitor_Price', 'resolved_highest_competitor_price',
       'market_position', 'Units Ordered', 'Demand Strength',
       'Conversion Rate', 'Revenue per Session', 'return_rate_90d',
       'return_risk', 'demand_return_regime', 'days-of-supply',
       'inventory_status', 'velocity_trend', 'clearance_candidate',
       'sell-through', 'pricing_strategy_inventory', 'ads_efficiency',
       'ad_dependency_flag', 'pricing_strategy_ads', 'price_lower_bound',
       'price_upper_bound', 'demand_return_score', 'inventory_score',
       'ads_score', 'final_pricing_score', 'price_adjustment_pct',
       'raw_recommended_price', 'recommended_price'],
      dtype='object')

In [56]:
master_df[['SKU', 'unit_cost', 'min_price', 'target_price', 'Current_Price','Demand Strength','demand_return_regime',
           'raw_recommended_price','recommended_price']]

,SKU,unit_cost,min_price,target_price,Current_Price,Demand Strength,demand_return_regime,raw_recommended_price,recommended_price
0,MN-01,32.30,40.375000,49.692308,38.90,Strong Demand,Mixed / Watch,40.845,40.845000
1,MN-02,26.60,33.250000,40.923077,33.90,Weak Demand,Mixed / Watch,35.595,35.595000
2,MN-03,30.03,37.537500,46.200000,34.90,Weak Demand,Mixed / Watch,36.645,37.537500
3,MN-04,25.94,32.425000,39.907692,29.90,Strong Demand,False Demand (High Returns),25.415,32.425000
4,MN-05,25.77,34.360000,46.854545,21.90,Weak Demand,Low Demand but Good Quality,21.900,34.360000
5,MN-06,14.37,19.160000,26.127273,14.95,Weak Demand,Low Demand but Good Quality,13.754,19.160000
6,MN-07,29.87,37.337500,45.953846,34.90,Weak Demand,Mixed / Watch,36.645,37.337500
7,MN-08,26.11,34.813333,47.472727,29.90,Weak Demand,Weak Demand & High Returns,25.415,34.813333
8,MN-09,14.92,19.893333,27.127273,14.90,Weak Demand,Weak Demand & High Returns,12.665,19.893333
9,MN-10,17.40,21.750000,26.769231,19.90,Weak Demand,Weak Demand & High Returns,16.915,21.750000
